# การทดลอง

## การทดลองที่ 1: การสร้าง B-Tree Node และ B-Tree

In [5]:
class BTreeNode:
    def __init__(self, leaf=True):
        # leaf: บอกว่าโหนดนี้เป็นใบหรือไม่
        self.leaf = leaf
        # keys: เก็บค่า key ในโหนด เรียงจากน้อยไปมาก
        self.keys = []
        # data: เก็บข้อมูลที่สัมพันธ์กับแต่ละ key
        self.data = []
        # children: เก็บ pointer ไปยังลูกของโหนด
        self.children = []

class BTree:
    def __init__(self, order):
        # root: pointer ไปยังโหนด root
        self.root = None
        # order: กำหนดจำนวน key สูงสุดในแต่ละโหนด
        self.order = order
        
    def get_min_keys(self):
        # คำนวณจำนวน key ขั้นต่ำที่แต่ละโหนดต้องมี
        return (self.order // 2) - 1 if self.order % 2 == 0 else self.order // 2
        
    def get_max_keys(self):
        # คำนวณจำนวน key สูงสุดที่แต่ละโหนดสามารถมีได้
        return self.order - 1

# การทดลอง

## การทดลองที่ 2: การเพิ่มข้อมูลใน B-Tree

In [6]:
import random

class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order):
        self.root = BTreeNode()
        self.order = order
        self.max_keys = order - 1
        self.min_keys = (order // 2) - 1 if order % 2 == 0 else order // 2
    
    def insert(self, key, data):
        root = self.root
        if len(root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False)
            new_root.children.append(root)
            self._split_child(new_root, 0)
            self.root = new_root
        
        self._insert_non_full(self.root, key, data)
    
    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            
            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            
            self._insert_non_full(node.children[i], key, data)
    
    def _split_child(self, parent, i):
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2
        
        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        
        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]
        
        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)
        
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
    
    def display(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, ":", node.keys)
        for child in node.children:
            self.display(child, level + 1)

#  order = 4
b_tree = BTree(order=4)

# เพิ่มข้อมูลนักศึกษา 5 คนด้วย ID แบบสุ่ม
random_ids = random.sample(range(100, 200), 5)
students = [(random_ids[i], name) for i, name in enumerate(["Ohem", "Mam", "Mew", "Phum", "Thun"])]
for id, name in students:
    b_tree.insert(id, name)

# แสดงโครงสร้างของ B-Tree
b_tree.display()

Level 0 : [141]
Level 1 : [125, 129]
Level 1 : [174, 198]


## ผลการทดลองที่ 2

### 1. เขียนโปรแกรมเพื่อทดสอบการค้นหาข้อมูลใน B-Tree ตามข้อมูลที่ได้เพิ่มในการทดลองก่อนหน้า

In [7]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order=3):
        self.root = None
        self.order = order

    def get_max_keys(self):
        return self.order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.get_max_keys():
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.get_max_keys():
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2  

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]

        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]

    def print_tree(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, ":", node.keys)
        for child in node.children:
            self.print_tree(child, level + 1)


# เพิ่มข้อมูลนักศึกษา
students = [(111, "A"), (171, "B"), (177, "C"), (107, "Dd"), (170, "E")]

btree = BTree(order=3)
for student in students:
    btree.insert(student[0], student[1])

# แสดงโครงสร้างของ B-Tree
btree.print_tree()

Level 0 : [111, 171]
Level 1 : [107]
Level 1 : [170]
Level 1 : [177]


### 2. แก้ไข class B-Tree ให้มีการเก็บจำนวน Entry สูงสุด และต่ำสุด แทนการใช้ get_min_keys และ get_max_keys

In [8]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order=3):
        self.root = None
        self.order = order
        self.max_keys = order - 1
        self.min_keys = (order // 2) - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2  # Adjusted to avoid IndexError

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]

        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]

    def print_tree(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, ":", node.keys)
        for child in node.children:
            self.print_tree(child, level + 1)


# เพิ่มข้อมูลนักศึกษา
students = [(111, "A"), (171, "B"), (177, "C"), (107, "Dd"), (170, "E")]

btree = BTree(order=3)
for student in students:
    btree.insert(student[0], student[1])

# แสดงโครงสร้างของ B-Tree
btree.print_tree()

Level 0 : [111, 171]
Level 1 : [107]
Level 1 : [170]
Level 1 : [177]


## การทดลองที่ 3: การค้นหาข้อมูลใน B-Tree

In [9]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order=3):
        self.root = None
        self.order = order
        self.max_keys = order - 1
        self.min_keys = (order // 2) - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2  # Adjusted to avoid IndexError

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]

        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]

    def search(self, key):
        """ค้นหาข้อมูลจาก key ที่กำหนด"""
        def _search_node(node, key):
            i = 0
            while i < len(node.keys) and key > node.keys[i]:
                i += 1
            if i < len(node.keys) and key == node.keys[i]:
                return (node, i)
            if node.leaf:
                return None
            return _search_node(node.children[i], key)

        if self.root is None:
            return None
        result = _search_node(self.root, key)
        if result:
            node, index = result
            return node.data[index]
        return None

    def print_tree(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, ":", node.keys)
        for child in node.children:
            self.print_tree(child, level + 1)


# เพิ่มข้อมูลนักศึกษา
students = [(111, "A"), (171, "B"), (177, "C"), (107, "Dd"), (170, "E")]

btree = BTree(order=3)
for student in students:
    btree.insert(student[0], student[1])

# แสดงโครงสร้างของ B-Tree
btree.print_tree()

# ทดสอบการค้นหา
search_key = 177
result = btree.search(search_key)
if result:
    print(f"พบข้อมูลของ {search_key}: {result}")
else:
    print(f"ไม่พบข้อมูลของ {search_key}")

Level 0 : [111, 171]
Level 1 : [107]
Level 1 : [170]
Level 1 : [177]
พบข้อมูลของ 177: C


## การทดลองที่ 4: การแสดงผล B-Tree

In [6]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order=3):
        self.root = None
        self.order = order
        self.max_keys = order - 1
        self.min_keys = (order // 2) - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2  # Adjusted to avoid IndexError

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]

        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]

    def search(self, key):
        """ค้นหาข้อมูลจาก key ที่กำหนด"""
        def _search_node(node, key):
            i = 0
            while i < len(node.keys) and key > node.keys[i]:
                i += 1
            if i < len(node.keys) and key == node.keys[i]:
                return (node, i)
            if node.leaf:
                return None
            return _search_node(node.children[i], key)

        if self.root is None:
            return None
        result = _search_node(self.root, key)
        if result:
            node, index = result
            return node.data[index]
        return None

    def print_tree(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, ":", node.keys)
        for child in node.children:
            self.print_tree(child, level + 1)

    def display(self):
        def _display(node, level):
            if node:
                print('  ' * level + f"Keys: {node.keys}")
                print('  ' * level + f"Data: {node.data}")
                print('  ' * level + f"Is Leaf: {node.leaf}")
                print('  ' * level + f"Number of children: {len(node.children)}")
                print()
                for child in node.children:
                    _display(child, level + 1)
        print("B-Tree Structure:")
        _display(self.root, 0)

# เพิ่มข้อมูลนักศึกษา
students = [(111, "Adam"), (171, "Max"), (177, "Choper"), (107, "DLufy"), (170, "Ergo")]
btree = BTree(order=3)
for student in students:
    btree.insert(student[0], student[1])

# แสดงโครงสร้างของ B-Tree
btree.print_tree()

# ทดสอบการค้นหา
search_key = 111
result = btree.search(search_key)
if result:
    print(f"พบข้อมูลของ {search_key}: {result}")
else:
    print(f"ไม่พบข้อมูลของ {search_key}")

# ทดสอบการแสดงผลข้อมูลใน B-Tree
btree.display()

Level 0 : [111, 171]
Level 1 : [107]
Level 1 : [170]
Level 1 : [177]
พบข้อมูลของ 111: Adam
B-Tree Structure:
Keys: [111, 171]
Data: ['Adam', 'Max']
Is Leaf: False
Number of children: 3

  Keys: [107]
  Data: ['DLufy']
  Is Leaf: True
  Number of children: 0

  Keys: [170]
  Data: ['Ergo']
  Is Leaf: True
  Number of children: 0

  Keys: [177]
  Data: ['Choper']
  Is Leaf: True
  Number of children: 0



## การทดลองที่ 5: ตัวอย่างการใช้งานจริง :ระบบทะเบียนนักศึกษา

In [10]:
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.children = []

class BTree:
    def __init__(self, order):
        self.root = BTreeNode(leaf=True)
        self.order = order  # กำหนดขนาดของ B-Tree (เช่น order = 3)

    def insert(self, key, value):
        root = self.root
        if len(root.keys) == (2 * self.order - 1):  # ถ้ารากเต็ม ต้องแยกโหนด
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self.split_child(new_root, 0)
            self.root = new_root
        self._insert_non_full(self.root, key, value)

    def _insert_non_full(self, node, key, value):
        if node.leaf:
            node.keys.append((key, value))
            node.keys.sort(key=lambda x: x[0])  # เรียงตามค่า key
        else:
            i = len(node.keys) - 1
            while i >= 0 and key < node.keys[i][0]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == (2 * self.order - 1):
                self.split_child(node, i)
                if key > node.keys[i][0]:
                    i += 1
            self._insert_non_full(node.children[i], key, value)

    def split_child(self, parent, i):
        order = self.order
        full_child = parent.children[i]
        new_child = BTreeNode(leaf=full_child.leaf)
        parent.keys.insert(i, full_child.keys[order - 1])
        new_child.keys = full_child.keys[order:]
        full_child.keys = full_child.keys[:order - 1]

        if not full_child.leaf:
            new_child.children = full_child.children[order:]
            full_child.children = full_child.children[:order]

        parent.children.insert(i + 1, new_child)

    def search(self, key):
        return self._search(self.root, key)

    def _search(self, node, key):
        i = 0
        while i < len(node.keys) and key > node.keys[i][0]:
            i += 1

        if i < len(node.keys) and key == node.keys[i][0]:
            return node.keys[i][1]

        if node.leaf:
            return None

        return self._search(node.children[i], key)

# สร้าง B-Tree สำหรับระบบทะเบียน
registration_system = BTree(order=3)

# เพิ่มข้อมูลนักศึกษา
def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

# เพิ่มข้อมูลตัวอย่าง
register_student(6301, {
    "name": "Perapat Ohm",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "Supattra Mam",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

# ค้นหาข้อมูลนักศึกษา
def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

# ทดสอบค้นหา
get_student_info(6301)

get_student_info(6302)

get_student_info(9999)  # ทดสอบค้นหานักศึกษาที่ไม่มีในระบบ


รหัสนักศึกษา: 6301
ชื่อ: Perapat Ohm
เกรดเฉลี่ย: 3.75
วิชาที่ลงทะเบียน: CS101, CS102
รหัสนักศึกษา: 6302
ชื่อ: Supattra Mam
เกรดเฉลี่ย: 3.85
วิชาที่ลงทะเบียน: CS101, MATH101
ไม่พบข้อมูลนักศึกษารหัส 9999


# แบบฝึกหัด

## 1. ให้นักศึกษาเพิ่มเมธอดสำหรับลบข้อมูล (key และ data) ออกจาก B-Tree

In [16]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order=3):
        self.root = None
        self.order = order
        self.max_keys = order - 1
        self.min_keys = (order // 2) - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]

        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]

    def search(self, key):
        def _search_node(node, key):
            i = 0
            while i < len(node.keys) and key > node.keys[i]:
                i += 1
            if i < len(node.keys) and key == node.keys[i]:
                return (node, i)
            if node.leaf:
                return None
            return _search_node(node.children[i], key)

        if self.root is None:
            return None
        result = _search_node(self.root, key)
        if result:
            node, index = result
            return node.data[index]
        return None

    def delete(self, key):
        def _delete_node(node, key):
            if node is None:
                return None

            if key in node.keys:
                index = node.keys.index(key)
                if node.leaf:
                    node.keys.pop(index)
                    node.data.pop(index)
                else:
                    node.keys[index] = node.children[index].keys[-1]
                    node.data[index] = node.children[index].data[-1]
                    _delete_node(node.children[index], node.keys[index])
            else:
                for i in range(len(node.keys)):
                    if key < node.keys[i]:
                        _delete_node(node.children[i], key)
                        return
                _delete_node(node.children[-1], key)
        
        _delete_node(self.root, key)

registration_system = BTree(order=3)

def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

register_student(6301, {
    "name": "Mama Coco",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "Black Panther",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

get_student_info(6301)

# ทดสอบลบข้อมูล
print("\nลบนักศึกษา 6301")
registration_system.delete(6301)
get_student_info(6301)

รหัสนักศึกษา: 6301
ชื่อ: Mama Coco
เกรดเฉลี่ย: 3.75
วิชาที่ลงทะเบียน: CS101, CS102

ลบนักศึกษา 6301
ไม่พบข้อมูลนักศึกษารหัส 6301


## 2. ให้นักศึกษาเพิ่มเมธอดสำหรับอัปเดตข้อมูล (data) สำหรับ key ที่กำหนด

In [17]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order=3):
        self.root = None
        self.order = order
        self.max_keys = order - 1
        self.min_keys = (order // 2) - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]

        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]

    def search(self, key):
        def _search_node(node, key):
            i = 0
            while i < len(node.keys) and key > node.keys[i]:
                i += 1
            if i < len(node.keys) and key == node.keys[i]:
                return (node, i)
            if node.leaf:
                return None
            return _search_node(node.children[i], key)

        if self.root is None:
            return None
        result = _search_node(self.root, key)
        if result:
            node, index = result
            return node.data[index]
        return None

    def delete(self, key):
        def _delete_node(node, key):
            if node is None:
                return None

            if key in node.keys:
                index = node.keys.index(key)
                if node.leaf:
                    node.keys.pop(index)
                    node.data.pop(index)
                else:
                    node.keys[index] = node.children[index].keys[-1]
                    node.data[index] = node.children[index].data[-1]
                    _delete_node(node.children[index], node.keys[index])
            else:
                for i in range(len(node.keys)):
                    if key < node.keys[i]:
                        _delete_node(node.children[i], key)
                        return
                _delete_node(node.children[-1], key)
        
        _delete_node(self.root, key)
    
    def update(self, key, new_data):
        result = self.search(key)
        if result:
            def _update_node(node, key, new_data):
                i = 0
                while i < len(node.keys) and key > node.keys[i]:
                    i += 1
                if i < len(node.keys) and key == node.keys[i]:
                    node.data[i] = new_data
                    return True
                if node.leaf:
                    return False
                return _update_node(node.children[i], key, new_data)
            _update_node(self.root, key, new_data)

registration_system = BTree(order=3)

def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

register_student(6301, {
    "name": "Mammose",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "Iron man",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

get_student_info(6301)

# ทดสอบลบข้อมูล
print("\nลบนักศึกษา 6301")
registration_system.delete(6301)
get_student_info(6301)

# ทดสอบอัปเดตข้อมูล
print("\nอัปเดตข้อมูลนักศึกษา 6302")
registration_system.update(6302, {
    "name": "สมหญิง เก่งมาก",
    "gpa": 4.00,
    "courses": ["CS101", "MATH101", "PHYS101"]
})
get_student_info(6302)

รหัสนักศึกษา: 6301
ชื่อ: Mammose
เกรดเฉลี่ย: 3.75
วิชาที่ลงทะเบียน: CS101, CS102

ลบนักศึกษา 6301
ไม่พบข้อมูลนักศึกษารหัส 6301

อัปเดตข้อมูลนักศึกษา 6302
รหัสนักศึกษา: 6302
ชื่อ: สมหญิง เก่งมาก
เกรดเฉลี่ย: 4.0
วิชาที่ลงทะเบียน: CS101, MATH101, PHYS101


## 3. ให้นักศึกษาเพิ่มเมธอดสำหรับแสดงข้อมูลทั้งหมดใน B-Tree เรียงตาม key

In [18]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order=3):
        self.root = None
        self.order = order
        self.max_keys = order - 1
        self.min_keys = (order // 2) - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]

        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]

    def search(self, key):
        def _search_node(node, key):
            i = 0
            while i < len(node.keys) and key > node.keys[i]:
                i += 1
            if i < len(node.keys) and key == node.keys[i]:
                return (node, i)
            if node.leaf:
                return None
            return _search_node(node.children[i], key)

        if self.root is None:
            return None
        result = _search_node(self.root, key)
        if result:
            node, index = result
            return node.data[index]
        return None

    def delete(self, key):
        def _delete_node(node, key):
            if node is None:
                return None

            if key in node.keys:
                index = node.keys.index(key)
                if node.leaf:
                    node.keys.pop(index)
                    node.data.pop(index)
                else:
                    node.keys[index] = node.children[index].keys[-1]
                    node.data[index] = node.children[index].data[-1]
                    _delete_node(node.children[index], node.keys[index])
            else:
                for i in range(len(node.keys)):
                    if key < node.keys[i]:
                        _delete_node(node.children[i], key)
                        return
                _delete_node(node.children[-1], key)
        
        _delete_node(self.root, key)
    
    def update(self, key, new_data):
        result = self.search(key)
        if result:
            def _update_node(node, key, new_data):
                i = 0
                while i < len(node.keys) and key > node.keys[i]:
                    i += 1
                if i < len(node.keys) and key == node.keys[i]:
                    node.data[i] = new_data
                    return True
                if node.leaf:
                    return False
                return _update_node(node.children[i], key, new_data)
            _update_node(self.root, key, new_data)

    def display_sorted(self):
        def _inorder_traversal(node):
            if node is not None:
                for i in range(len(node.keys)):
                    if not node.leaf:
                        _inorder_traversal(node.children[i])
                    print(f"รหัสนักศึกษา: {node.keys[i]}, ข้อมูล: {node.data[i]}")
                if not node.leaf:
                    _inorder_traversal(node.children[-1])
        _inorder_traversal(self.root)

registration_system = BTree(order=3)

def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

register_student(6301, {
    "name": "Somsak",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "somsri",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

get_student_info(6301)

print("\nลบนักศึกษา 6301")
registration_system.delete(6301)
get_student_info(6301)

print("\nอัปเดตข้อมูลนักศึกษา 6302")
registration_system.update(6302, {
    "name": "สมหญิง เก่งมาก",
    "gpa": 4.00,
    "courses": ["CS101", "MATH101", "PHYS101"]
})
get_student_info(6302)

print("\nแสดงข้อมูลนักศึกษาทั้งหมดเรียงตามรหัสนักศึกษา:")
registration_system.display_sorted()

รหัสนักศึกษา: 6301
ชื่อ: Somsak
เกรดเฉลี่ย: 3.75
วิชาที่ลงทะเบียน: CS101, CS102

ลบนักศึกษา 6301
ไม่พบข้อมูลนักศึกษารหัส 6301

อัปเดตข้อมูลนักศึกษา 6302
รหัสนักศึกษา: 6302
ชื่อ: สมหญิง เก่งมาก
เกรดเฉลี่ย: 4.0
วิชาที่ลงทะเบียน: CS101, MATH101, PHYS101

แสดงข้อมูลนักศึกษาทั้งหมดเรียงตามรหัสนักศึกษา:
รหัสนักศึกษา: 6302, ข้อมูล: {'name': 'สมหญิง เก่งมาก', 'gpa': 4.0, 'courses': ['CS101', 'MATH101', 'PHYS101']}


## 4. ให้นักศึกษาเพิ่มเมธอดสำหรับค้นหาข้อมูลแบบช่วง (range search)

In [19]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []

class BTree:
    def __init__(self, order=3):
        self.root = None
        self.order = order
        self.max_keys = order - 1
        self.min_keys = (order // 2) - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = len(child.keys) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])
        parent.children.insert(i + 1, new_node)

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]

        if not child.leaf:
            new_node.children = child.children[mid + 1:]
            child.children = child.children[:mid + 1]

    def search(self, key):
        def _search_node(node, key):
            i = 0
            while i < len(node.keys) and key > node.keys[i]:
                i += 1
            if i < len(node.keys) and key == node.keys[i]:
                return (node, i)
            if node.leaf:
                return None
            return _search_node(node.children[i], key)

        if self.root is None:
            return None
        result = _search_node(self.root, key)
        if result:
            node, index = result
            return node.data[index]
        return None

    def delete(self, key):
        def _delete_node(node, key):
            if node is None:
                return None

            if key in node.keys:
                index = node.keys.index(key)
                if node.leaf:
                    node.keys.pop(index)
                    node.data.pop(index)
                else:
                    node.keys[index] = node.children[index].keys[-1]
                    node.data[index] = node.children[index].data[-1]
                    _delete_node(node.children[index], node.keys[index])
            else:
                for i in range(len(node.keys)):
                    if key < node.keys[i]:
                        _delete_node(node.children[i], key)
                        return
                _delete_node(node.children[-1], key)
        
        _delete_node(self.root, key)
    
    def update(self, key, new_data):
        result = self.search(key)
        if result:
            def _update_node(node, key, new_data):
                i = 0
                while i < len(node.keys) and key > node.keys[i]:
                    i += 1
                if i < len(node.keys) and key == node.keys[i]:
                    node.data[i] = new_data
                    return True
                if node.leaf:
                    return False
                return _update_node(node.children[i], key, new_data)
            _update_node(self.root, key, new_data)

    def display_sorted(self):
        def _inorder_traversal(node):
            if node is not None:
                for i in range(len(node.keys)):
                    if not node.leaf:
                        _inorder_traversal(node.children[i])
                    print(f"รหัสนักศึกษา: {node.keys[i]}, ข้อมูล: {node.data[i]}")
                if not node.leaf:
                    _inorder_traversal(node.children[-1])
        _inorder_traversal(self.root)
    
    def range_search(self, low, high):
        result = []
        def _range_search(node):
            if node is not None:
                for i in range(len(node.keys)):
                    if node.keys[i] >= low and node.keys[i] <= high:
                        result.append((node.keys[i], node.data[i]))
                    if not node.leaf:
                        _range_search(node.children[i])
                if not node.leaf:
                    _range_search(node.children[-1])
        _range_search(self.root)
        return result

registration_system = BTree(order=3)

def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

register_student(6301, {
    "name": "Goku",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "Luffy",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

get_student_info(6301)

print("\nค้นหานักศึกษาภายในช่วง 6300 - 6310")
print(registration_system.range_search(6300, 6310))

รหัสนักศึกษา: 6301
ชื่อ: Goku
เกรดเฉลี่ย: 3.75
วิชาที่ลงทะเบียน: CS101, CS102

ค้นหานักศึกษาภายในช่วง 6300 - 6310
[(6301, {'name': 'Goku', 'gpa': 3.75, 'courses': ['CS101', 'CS102']}), (6302, {'name': 'Luffy', 'gpa': 3.85, 'courses': ['CS101', 'MATH101']})]
